In [45]:
%%writefile server_kernel.py

from ipykernel.kernelbase import Kernel
import numpy as np
import os
import matplotlib.pyplot as plt
from io import BytesIO
import urllib, base64
import socket

Overwriting server_kernel.py


In [46]:
%%writefile server_kernel.py -a

class server_kernel(Kernel):
    implementation = 'server'
    implementation_version = '1.0'
    language = 'python'  # will be used for
                         # syntax highlighting
    language_version = '3.6'
    language_info = {'name': 'server_kernel',
                     'mimetype': 'text/plain',
                     'extension': '.py'}
    banner = "Simple client"

Appending to server_kernel.py


In [47]:
%%writefile server_kernel.py -a

    def do_execute(self, code, silent,
                   store_history=True,
                   user_expressions=None,
                   allow_stdin=False):

        port= code.split('\n')[0]
        file_name = code.split('\n')[1]
        
        
        s = socket.socket()         # Create a socket object
        host = socket.gethostname() # Get local machine name
        port = int(port)           # Reserve a port for your service.
        s.bind((host, port))        # Bind to the port
        
        
        s.listen(5)  
        result=""
        # Now wait for client connection.
        while True:
            c, addr = s.accept()     # Establish connection with client.
            result += str('Got connection from')+str(addr)
            
            file1 = open(file_name,"r+") 
            data=file1.read()
            c.send(data.encode())
            
            c.close()   
        
        if not silent:
            # We send the standard output to the
            # client.
            self.send_response(
                self.iopub_socket,
                'stream', {
                    'name': 'stdout',
                    'data': ('Plotting {n} ')})

            # We prepare the response with our rich
            # data (the plot).
            content = {
                'source': 'kernel',

                # This dictionary may contain
                # different MIME representations of
                # the output.
                'data': {
                        'text/plain' : result
                },

                
            }

            # We send the display_data message with
            # the contents.
            self.send_response(self.iopub_socket,
                'display_data', content)

        # We return the exection results.
        return {'status': 'ok',
                'execution_count':
                    self.execution_count,
                'payload': [],
                'user_expressions': {},
               }

Appending to server_kernel.py


In [48]:
%%writefile server_kernel.py -a

if __name__ == '__main__':
    from ipykernel.kernelapp import IPKernelApp
    IPKernelApp.launch_instance(
        kernel_class=server_kernel)

Appending to server_kernel.py


In [49]:
%mkdir -p server_kernel/

In [1]:
%%writefile server_kernel/kernel.json
{
 "argv": ["python", "-m",
          "server_kernel", "-f",
          "{connection_file}"],
 "display_name": "ServerKernel",
 "name": "ServerKernel",
 "language": "python"
}

Overwriting server_kernel/kernel.json


In [2]:
!jupyter kernelspec install --user server_kernel

[InstallKernelSpec] Removing existing kernelspec in /Users/anshulkulhari/Library/Jupyter/kernels/server_kernel
[InstallKernelSpec] Installed kernelspec server_kernel in /Users/anshulkulhari/Library/Jupyter/kernels/server_kernel


In [3]:
!jupyter kernelspec list

Available kernels:
  python3          /usr/local/Cellar/jupyterlab/3.0.14/libexec/lib/python3.9/site-packages/ipykernel/resources
  client_kernel    /Users/anshulkulhari/Library/Jupyter/kernels/client_kernel
  githubkernel     /Users/anshulkulhari/Library/Jupyter/kernels/githubkernel
  grammar          /Users/anshulkulhari/Library/Jupyter/kernels/grammar
  plag_files       /Users/anshulkulhari/Library/Jupyter/kernels/plag_files
  plotter          /Users/anshulkulhari/Library/Jupyter/kernels/plotter
  server_kernel    /Users/anshulkulhari/Library/Jupyter/kernels/server_kernel
